In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
class FeatureSelectionLayer(tf.keras.layers.Layer):
    """ 
    A Layer that creates a one-to-one connection between the input neurons and the output neurons.
    The weights are trainable, so the layer can be used as a feature selection layer.
    
    Possible usage: e.g. gene expression data
    """  
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def build(self, input_shape):
        self.kernel = self.add_weight("kernel", shape=(int(input_shape[-1]),))
    
    def call(self, inputs):
        return tf.multiply(inputs, self.kernel)

### Example usage (Genexpressionanalysis with Single-Cell RNASeq-Data):
Here i used a smaller test data set of some gene expression data, to predict if somone is a smoker or not. 
The problem: We have 18487 Feature (gene expression value). 

In [3]:
df_train = pd.read_csv("./train.csv")
df_test = pd.read_csv("./test.csv")
df_val = pd.read_csv("./val.csv")

In [4]:
X_train = df_train.drop(columns=['SmokCigNow','ATS_PackYears'], axis = 1).to_numpy().astype('float32')
Y_train = df_train['SmokCigNow'].to_numpy().reshape(-1,1)

In [5]:
X_val = df_val.drop(columns=['SmokCigNow','ATS_PackYears'], axis = 1).to_numpy().astype('float32')
Y_val = df_val['SmokCigNow'].to_numpy().reshape(-1,1)
X_test = df_test.drop(columns=['SmokCigNow', 'ATS_PackYears'], axis = 1).to_numpy().astype('float32')
Y_test = df_test['SmokCigNow'].to_numpy().reshape(-1,1)

### Without feature selection layer

In [6]:

actf = 'relu'
input_shape = X_train.shape[1]
kernel_regularizer= tf.keras.regularizers.L2(0.015)
kernel_regularizer_2= tf.keras.regularizers.L2(0.005)

mdl = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=actf, kernel_regularizer=kernel_regularizer),
    tf.keras.layers.Dense(2, activation='softmax'),
                          ])

loss_func = tf.keras.losses.SparseCategoricalCrossentropy()

mdl.compile(optimizer='adam',
            loss=loss_func,
            metrics=['accuracy'])

mdl.fit(X_train,
        Y_train,
        validation_data=(X_val,Y_val),
        batch_size=16,
        epochs=10,
        verbose=1)

Epoch 1/10


2022-07-18 19:47:14.259399: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


7/7 [==============================] - 1s 83ms/step - loss: 156.9794 - accuracy: 0.5000 - val_loss: 52.5458 - val_accuracy: 0.8333
Epoch 2/10
7/7 [==============================] - 0s 56ms/step - loss: 43.8273 - accuracy: 0.5400 - val_loss: 10.4182 - val_accuracy: 0.8000
Epoch 3/10
7/7 [==============================] - 0s 58ms/step - loss: 28.4813 - accuracy: 0.5500 - val_loss: 23.9411 - val_accuracy: 0.8333
Epoch 4/10
7/7 [==============================] - 0s 65ms/step - loss: 39.3054 - accuracy: 0.6400 - val_loss: 11.6254 - val_accuracy: 0.8333
Epoch 5/10
7/7 [==============================] - 0s 57ms/step - loss: 27.9112 - accuracy: 0.6100 - val_loss: 15.7829 - val_accuracy: 0.8333
Epoch 6/10
7/7 [==============================] - 0s 57ms/step - loss: 32.4452 - accuracy: 0.5400 - val_loss: 11.3398 - val_accuracy: 0.8333
Epoch 7/10
7/7 [==============================] - 0s 58ms/step - loss: 16.1408 - accuracy: 0.7100 - val_loss: 11.4933 - val_accuracy: 0.8333
Epoch 8/10
7/7 [=======

### With feature selection layer

In [7]:

actf = 'relu'
input_shape = X_train.shape[1]
kernel_regularizer= tf.keras.regularizers.L2(0.015)
kernel_regularizer_2= tf.keras.regularizers.L2(0.005)

mdl = tf.keras.models.Sequential([
    FeatureSelectionLayer(name="feature_selection_input", input_shape=(input_shape,)),
    tf.keras.layers.Dense(512, activation=actf, kernel_regularizer=kernel_regularizer),
    tf.keras.layers.Dense(2, activation='softmax'),
                          ])

loss_func = tf.keras.losses.SparseCategoricalCrossentropy()

mdl.compile(optimizer='adam',
            loss=loss_func,
            metrics=['accuracy'])

mdl.fit(X_train,
        Y_train,
        validation_data=(X_val,Y_val),
        batch_size=16,
        epochs=10,
        verbose=1)

Epoch 1/10
7/7 [==============================] - 1s 77ms/step - loss: 11.1186 - accuracy: 0.6300 - val_loss: 5.3596 - val_accuracy: 0.1667
Epoch 2/10
7/7 [==============================] - 0s 59ms/step - loss: 3.5327 - accuracy: 0.5400 - val_loss: 1.6241 - val_accuracy: 0.8333
Epoch 3/10
7/7 [==============================] - 0s 58ms/step - loss: 1.4666 - accuracy: 0.7000 - val_loss: 1.0749 - val_accuracy: 0.8333
Epoch 4/10
7/7 [==============================] - 0s 59ms/step - loss: 1.3527 - accuracy: 0.6800 - val_loss: 1.1939 - val_accuracy: 0.7333
Epoch 5/10
7/7 [==============================] - 0s 60ms/step - loss: 1.0458 - accuracy: 0.7800 - val_loss: 0.7567 - val_accuracy: 0.8333
Epoch 6/10
7/7 [==============================] - 0s 60ms/step - loss: 0.8614 - accuracy: 0.6800 - val_loss: 0.6597 - val_accuracy: 0.8333
Epoch 7/10
7/7 [==============================] - 0s 59ms/step - loss: 0.6882 - accuracy: 0.6800 - val_loss: 0.5930 - val_accuracy: 0.8333
Epoch 8/10
7/7 [==========